# VAE

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
# from torchvision import datasets, transforms
import dataset
import mainmodel
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.svm import OneClassSVM
from sklearn.metrics import roc_curve

## 学習データ作成

In [12]:

Dataset = dataset.dataset("Obrid_AE", "data")
Dataset.concat_data("sample_data",500)
Dataset = dataset.dataset("Obrid_AE", "test")
print("----------------------")
Dataset.concat_data("sample_test",100)
print("----------------------")
data = Dataset.read_savedata("sample_test")
print(data.shape[0])
print("----------------------")
data, test_data , anomaly_data= Dataset.read_traindata("sample_data", "sample_test", 1000, 256, 1)


(500, 256)
(500, 256)
----------------------
(100, 256)
(100, 256)
----------------------
(100, 256)
100
----------------------
(500, 256)
(100, 256)
1148 0
rate 0.9
data.shape[0]: 500
rate 450
TrainData (1000, 256, 1, 1, 256)
TestData (50, 256)
ÄnomalyDta (100, 256)


## モデル作成

![](https://standardfrancis.files.wordpress.com/2015/04/screenshot-from-2015-04-16-133436.png?w=1008)

In [7]:
class BatchNorm(nn.Module):
    def __init__(self, shape, epsilon=np.float32(1e-5)):
        super().__init__()
        self.gamma = nn.Parameter(torch.tensor(np.ones(shape, dtype='float32')))
        self.beta = nn.Parameter(torch.tensor(np.zeros(shape, dtype='float32')))
        self.epsilon = epsilon

    def forward(self, x):
        mean = torch.mean(x, (0, 2, 3), keepdim=True)
        std = torch.std(x, (0, 2, 3), keepdim=True)
        x_normalized = (x - mean) / (std**2 + self.epsilon)**0.5
        return self.gamma * x_normalized + self.beta

In [27]:
# torch.log(0)によるnanを防ぐ
def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))


class AE(nn.Module):
    def __init__(self, z_dim):
        super(AE, self).__init__()
        self.enc1 = nn.Conv1d(in_channels = 1, out_channels = 1, kernel_size = 7, stride = 5)
        self.enc2 = nn.Conv1d(in_channels = 1, out_channels = 1, kernel_size = 5, stride = 3)
        self.flat = nn.Flatten(0, -1)
        self.enc3 = nn.Linear(16, z_dim)

        self.encmean = nn.Linear(16, z_dim)
        self.encvar = nn.Linear(16, z_dim)

        # self.dec1 = nn.Linear(z_dim, 200)
        self.dec1 = nn.Linear(z_dim,  16)
        self.unflat = nn.Unflatten(0, (1, 1 ,16))
        self.dec2 = nn.ConvTranspose1d(in_channels = 1, out_channels = 1, kernel_size = 5, stride = 3, output_padding=1)
        self.dec3 = nn.ConvTranspose1d(in_channels = 1, out_channels = 1, kernel_size = 7, stride = 5, padding = 1, output_padding=1)


    def _encoder(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = self.flat(x)
        mean = self.encmean(x)
        var = F.softplus(self.encvar(x))
        return mean, var

    def _sample_z(self, mean, var):
        epsilon = torch.randn(mean.shape).to(device)
        return mean + torch.sqrt(var) * epsilon

    def _decoder(self, z):
        x = F.relu(self.dec1(z))
        x = self.unflat(x)
        x = F.relu(self.dec2(x))
        x = torch.sigmoid(self.dec3(x))
        return x

    def forward(self, x):
        mean, var = self._encoder(x)
        z = self._sample_z(mean, var)
        x = self._decoder(z)
        return x, z

    def loss(self, x):
        mean, var = self._encoder(x)
        # print(mean, var)
        # KL lossの計算
        KL = -0.5 * torch.mean(torch.sum(1 + torch_log(var) - mean**2 - var, dim=0))
        
        z = self._sample_z(mean, var)
        y = self._decoder(z)

        # reconstruction lossの計算
        reconstruction = torch.mean(torch.sum(x * torch_log(y) + (1 - x) * torch_log(1 - y), dim=0))

        return KL, -reconstruction 

## モデル作成

## モデル学習

In [31]:
z_dim = 3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AE(z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
input_data = torch.from_numpy(data.astype(np.float32)).clone()
input_test_data = torch.from_numpy(test_data.astype(np.float32)).clone()
for epoch in input_data:
    losses = []
    KL_losses = []
    reconstruction_losses = []
    model.train()
    for x in epoch:
        model.zero_grad()
        KL_loss, reconstruction_loss = model.loss(x)  # lossの各項の計算
        loss = KL_loss + reconstruction_loss  # 和を取ってlossとする
        loss.backward()
        optimizer.step()
        losses.append(loss.detach().numpy())
        KL_losses.append(KL_loss.detach().numpy())
        reconstruction_losses.append(reconstruction_loss.detach().numpy())

    losses_val = []
    model.eval()
    # for x in input_test_data:
    #     KL_loss, reconstruction_loss = model.loss(x)
    #     loss = KL_loss + reconstruction_loss
    #     losses_val.append(loss.cpu().detach().numpy())

    # print('EPOCH:%d, Train Lower Bound:%lf, (%lf, %lf), Valid Lower Bound:%lf' %
    #     (epoch+1, np.average(losses), np.average(KL_losses), np.average(reconstruction_losses)))

tensor([-0.0072,  0.0283,  0.1305], grad_fn=<AddBackward0>) tensor([0.6292, 0.6073, 0.7090], grad_fn=<SoftplusBackward0>)
tensor([-0.0062,  0.0273,  0.1295], grad_fn=<AddBackward0>) tensor([0.6297, 0.6078, 0.7095], grad_fn=<SoftplusBackward0>)
tensor([-0.0052,  0.0263,  0.1285], grad_fn=<AddBackward0>) tensor([0.6302, 0.6083, 0.7100], grad_fn=<SoftplusBackward0>)
tensor([-0.0042,  0.0253,  0.1275], grad_fn=<AddBackward0>) tensor([0.6306, 0.6087, 0.7105], grad_fn=<SoftplusBackward0>)
tensor([-0.0033,  0.0243,  0.1265], grad_fn=<AddBackward0>) tensor([0.6311, 0.6092, 0.7110], grad_fn=<SoftplusBackward0>)
tensor([-0.0023,  0.0233,  0.1255], grad_fn=<AddBackward0>) tensor([0.6316, 0.6096, 0.7115], grad_fn=<SoftplusBackward0>)
tensor([-0.0015,  0.0224,  0.1245], grad_fn=<AddBackward0>) tensor([0.6320, 0.6101, 0.7121], grad_fn=<SoftplusBackward0>)
tensor([-0.0006,  0.0214,  0.1235], grad_fn=<AddBackward0>) tensor([0.6325, 0.6105, 0.7126], grad_fn=<SoftplusBackward0>)
tensor([1.1946e-04, 2.04

ValueError: only one element tensors can be converted to Python scalars

## モデルの保存

In [ ]:
# model_name = "newmodel"
# ModelEdit = mainmodel.Modeledit("syuron")
# ModelEdit.save_model(model, model_name) 